In [1]:
import pandas as pd
import os

---

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/Xtest.csv', usecols=['Password'])
target = train.Times.copy()

In [3]:
df = pd.concat([train, test], sort=False)

---

`Это соревнование - перезапуск вступительного inClass на тех же данных. Теперь новые правила - можно пользоваться репозиторием с паролями. Посмотрим на разрешенные файлы`

In [4]:
PATH = '../data2/external_data/SecLists-master/Passwords/Common-Credentials/'
os.listdir(PATH)

['top-passwords-shortlist.txt',
 '10-million-password-list-top-1000000.txt',
 '10-million-password-list-top-10000.txt',
 'common-passwords-win.txt',
 'top-20-common-SSH-passwords.txt',
 'SplashData-2015-2.txt',
 'medical-devices.txt',
 'SplashData-2015-1.txt',
 'best110.txt',
 '10-million-password-list-top-100000.txt',
 'worst-passwords-2017-top100-slashdata.txt',
 '10-million-password-list-top-100.txt',
 '10-million-password-list-top-500.txt',
 '10k-most-common.txt',
 '500-worst-passwords.txt',
 'best15.txt',
 'SplashData-2014.txt',
 'best1050.txt',
 '10-million-password-list-top-1000.txt',
 'four-digit-pin-codes-sorted-by-frequency-withcount.csv']

`Большая часть оказывается бесполезной, но обратим внимание на файлик 10-million-password-list-top-1000000.txt`

In [5]:
file = '10-million-password-list-top-1000000.txt'

with open(PATH + file, 'r', encoding='Windows-1251') as f:
    lines = [x.rstrip("\n") for x in f.readlines()]

In [6]:
leak = pd.DataFrame(lines, columns=['Password'])

`Посмотрим на датафрейм`

In [7]:
leak.head(10)

,Password
0,123456
1,password
2,12345678
3,qwerty
4,123456789
5,12345
6,1234
7,111111
8,1234567
9,dragon


`теперь посмотрим на отсортированный по таргету train`

In [9]:
df.sort_values('Times', ascending=False).head(10)

,Password,Times
2715397,123456,55893.0
3136279,qwerty,13137.0
1175081,123456789,11696.0
2363307,12345,10938.0
2988373,1234,6432.0
2307329,111111,5682.0
3348280,1234567,4796.0
1795496,dragon,3927.0
3336606,123123,3845.0
3798071,baseball,3565.0


`Хм, первые записи ликового файла очень похожи на отсортированный трейн. Видим знакомые 123456, qwerty и затем dragon - оказывается файлик не просто содержит пароли, но и ОТСОРТИРОВАН по частоте встречаемости`

`Делаем левый джоин`

In [10]:
tt = pd.merge(leak, df[['Password', 'Times']], on='Password', how='left')

In [11]:
tt.head(20)

,Password,Times
0,123456,55893.0
1,password,NaN
2,12345678,NaN
3,qwerty,13137.0
4,123456789,11696.0
5,12345,10938.0
6,1234,6432.0
7,111111,5682.0
8,1234567,4796.0
9,dragon,3927.0


`Там где на месте таргета Nan - пароли из тестовой выборки. Заполним пропуски как среднее значение строчки сверху и строчки снизу`

In [12]:
tt['leak_backfill'] = tt.Times.fillna(method='backfill')
tt['leak_ffill'] = tt.Times.fillna(method='ffill')

In [13]:
tt['leak_nearest_mean'] = tt['leak_backfill']*0.5 + tt['leak_ffill']*0.5

`Такое решение дает скор 0.00128 (топ-5)`

---

`Между тем осталось много паролей, у которых сосед сверху не совпадает с соседом снизу - это значение можно подкрутить`

`Количество сабмитов ограничено 5, но это соревнование запускалось дважды. Разбиение на паблик и прайват поменялось, но тестовые данные остались те же! Это значит что можно пробивать лидерборд нового соревнования, делая Late сабмишены в первом (старом)`

In [40]:
tt[tt.leak_backfill != tt.leak_ffill].shape

(330, 5)

`Оказалось всего лишь 330 строк, где верхняя строчка не совпала с нижней`

In [47]:
tt[tt.leak_backfill != tt.leak_ffill].tail(10)

,Password,Times,leak_backfill,leak_ffill,leak_nearest_mean
63565,zzz777,NaN,13.0,14.0,13.5
63566,zymurgy,NaN,13.0,14.0,13.5
72669,524524,NaN,12.0,11.0,11.5
81721,000000q,NaN,10.0,11.0,10.5
81722,ZZZZZZZZ,NaN,10.0,11.0,10.5
123787,car1,NaN,8.0,7.0,7.5
178958,anaan,NaN,6.0,5.0,5.5
343033,717400,NaN,2.0,3.0,2.5
369647,------,NaN,2.0,3.0,2.5
609736,caphorn,NaN,2.0,3.0,2.5


`Нам более интересны объекты с большой **относительной** разницей между строчками, потому что метрика RMSLE. Таким образом за примерно 20 сабмитов в старое соревнование можно получить правильные ответы для некоторых строк и снизить ошибку с 0.00128 до 0.00100 и ниже `

In [ ]:
test.loc[test.Password == 'password', 'result'] = 19000
test.loc[test.Password == '12345678', 'result'] = 13500
test.loc[test.Password == 'car1', 'result'] = 7
test.loc[test.Password == 'anaan', 'result'] = 5
test.loc[test.Password == '000000q', 'result'] = 11
test.loc[test.Password == 'ZZZZZZZZ', 'result'] = 11
test.loc[test.Password == '717400', 'result'] = 3
test.loc[test.Password == '------', 'result'] = 3
test.loc[test.Password == 'caphorn', 'result'] = 2